<a href="https://colab.research.google.com/github/nobuhikosekiya/qiitablog_vectorsearch_elasticsearch/blob/main/qiitablog_vectorsearch_elasticsearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# このJupyter Notebookの使い方
これはレシピ集として作成されています。最初に、初期設定セクションにて利用するElasticsearch環境やOpenAI環境、HuggingFace環境の接続情報を設定します。その後は、実行したいセクションA. B. C. ..から始めて順番にコマンドを実行してください。
そのためにセクション間で重複したコードが繰り返しあります。

# 1.初期設定

## ライブラリの有効化

In [1]:
%pip install -q elasticsearch==8.11.1 langchain==0.0.352  openai==1.6.1 tiktoken jq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.8/412.8 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 794.4/794.4 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 656.0/656.0 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.8/59.8 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.4/192.4 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from pprint import pprint
import os, time
from getpass import getpass
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
from langchain.vectorstores.elasticsearch import ElasticsearchStore
from langchain.embeddings.openai import OpenAIEmbeddings

## Elasticsearchの設定

In [4]:
ELASTIC_CLOUD_ID = getpass("Elastic deployment Cloud ID")
ELASTIC_API_KEY = getpass("Elastic deployment API Key")
if ELASTIC_CLOUD_ID == '':
  ELASTIC_URL = getpass("Elastic deployment URL. No need if Cloud ID is provided.")
if ELASTIC_API_KEY == '':
  ELASTIC_USER = getpass("Elastic user. No need if API key is provided.")
  ELASTIC_PASSWORD = getpass("Elastic password. No need if API key is provided.")

if ELASTIC_CLOUD_ID != '' and ELASTIC_API_KEY != '':
  es = Elasticsearch(
    cloud_id=ELASTIC_CLOUD_ID,
    api_key=ELASTIC_API_KEY,
    request_timeout=300
  )
elif ELASTIC_URL != '' and ELASTIC_USER != '' and ELASTIC_PASSWORD != '':
  es = Elasticsearch(
    hosts = ELASTIC_URL,
    basic_auth=(ELASTIC_USER, ELASTIC_PASSWORD),
    request_timeout=300
  )
elif ELASTIC_URL != '' and ELASTIC_USER == '':
  es = Elasticsearch(
    hosts = ELASTIC_URL,
    # request_timeout=300,
    request_timeout=300
  )
else:
  print("env needs to set either ELASTIC_CLOUD_ID or ELASTIC_URL")


pprint(es.info()) # should return cluster info

Elastic deployment Cloud ID··········
Elastic deployment API Key··········
ObjectApiResponse({'name': 'instance-0000000036', 'cluster_name': '507a2cf6ba204071943512e0537eee58', 'cluster_uuid': 'oF-xDLtXRCet87gRuM3eJg', 'version': {'number': '8.11.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '76013fa76dcbf144c886990c6290715f5dc2ae20', 'build_date': '2023-12-05T10:03:47.729926671Z', 'build_snapshot': False, 'lucene_version': '9.8.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})


## テスト名の設定

In [5]:
TEST_NAME=input("Set the test name (Optional):")

if TEST_NAME == '':
    INDEX_PREFIX='test'
else:
    INDEX_PREFIX=TEST_NAME
INDEX_PREFIX

Set the Elasticsearch index name prefix (Optional):test1226


'test1226'

## OpenAIかAzure OpenAIの有効化

In [6]:
num = input("Enter number (or nothing to skip): 1: OpenAI 2: Azure OpenAI ")
USE_OPENAI = num == "1"
USE_AZURE_OPENAI = num == "2"
os.environ["OPENAI_API_KEY"] = ""

Enter number (or nothing to skip): 1: OpenAI 2: Azure OpenAI 1


### OpenAI APIの有効化

In [7]:
if USE_OPENAI:
  os.environ["OPENAI_API_KEY"] = getpass("OpenAI API key: ")
  # os.environ["OPENAI_API_TYPE"] = "open_ai"
  # os.environ["OPENAI_API_BASE"] = "https://api.openai.com/v1"

OpenAI API key: ··········


### Azure OpenAI APIの設定

In [8]:
if USE_AZURE_OPENAI:
    os.environ["OPENAI_API_BASE"] = getpass("Azure OpenAI API endpoint: ")
    os.environ["OPENAI_API_KEY"] = getpass("Azure OpenAI API key: ")
    os.environ["OPENAI_API_TYPE"] = "azure"
    os.environ["OPENAI_API_VERSION"] = "2023-05-15"
    azureOpenAI_embedding_deployment = getpass("Name of Azure OpenAI deployment for embedding")
    azureOpenAI_chat_deployment = getpass("Name of Azure OpenAI deployment for chat")

## Elastic 上のMLモデルの有効化

### 有効化するMLモデルの設定

In [10]:
enable_multilinguale5small = input("Enable ML model intfloat/multilingual-e5-small? Enter any key for 'Yes'.") != ""
enable_bertbasejapanesev3 = input("Enable ML model cl-tohoku/bert-base-japanese-v3? Enter any key for 'Yes'.") != ""
enable_bertbasejapanesev2 = input("Enable ML model cl-tohoku/bert-base-japanese-v2? Enter any key for 'Yes'.") != ""

Enable ML model intfloat/multilingual-e5-small? Enter any key for 'Yes'.··········
Enable ML model cl-tohoku/bert-base-japanese-v3? Enter any key for 'Yes'.··········
Enable ML model cl-tohoku/bert-base-japanese-v2? Enter any key for 'Yes'.··········


In [11]:
%pip install -q eland elasticsearch transformers sentence_transformers
import tempfile
from eland.ml.pytorch import PyTorchModel
from eland.ml.pytorch.transformers import TransformerModel
from eland.ml.pytorch.transformers import elasticsearch_model_id

def load_model(es_connection, model_id, task_type):
  with tempfile.TemporaryDirectory() as tmp_dir:
    print(f"Loading HuggingFace transformer tokenizer and model [{model_id}] for task [{task_type}]" )

    tm = TransformerModel(model_id=model_id, task_type=task_type)
    model_path, config, vocab_path = tm.save(tmp_dir)

    ptm = PyTorchModel(es_connection, tm.elasticsearch_model_id())
    model_exists = es_connection.options(ignore_status=404).ml.get_trained_models(model_id=ptm.model_id).meta.status == 200

    if model_exists:
      print("Model has already been imported")
    else:
      print("Importing model")
      ptm.import_model(model_path=model_path, config_path=None, vocab_path=vocab_path, config=config)
      print(f"Model successfully imported with id '{ptm.model_id}'")

def start_model(es_connection, model_id=None, es_model_id=None):
    if model_id:
      es_model_id = elasticsearch_model_id(model_id)
    if is_model_started(es_connection, es_model_id=es_model_id):
      print(f"Model '{es_model_id}' is already started")
    else:
      print(f"Starting model deployment '{es_model_id}'")
      es_connection.options(request_timeout=300).ml.start_trained_model_deployment(
            model_id=es_model_id, timeout="300s", wait_for="started")
      print(f"Model successfully started with id '{es_model_id}'")

def stop_model(es_connection, model_id=None, es_model_id=None):
    if model_id:
      es_model_id = elasticsearch_model_id(model_id)
    es_connection.options(request_timeout=300).ml.stop_trained_model_deployment(model_id=es_model_id, force=True)
    print("Stopping model deployment")

def is_model_started(es_connection, model_id=None, es_model_id=None):
    if model_id:
      es_model_id = elasticsearch_model_id(model_id)
    r = es_connection.ml.get_trained_models_stats(model_id=es_model_id)
    if r['trained_model_stats'][0].get('deployment_stats') != None:
      state = r['trained_model_stats'][0]['deployment_stats']['state']
      if state == 'started':
        print("Check: Model is started")
        return True
      elif state == 'starting':
        print("Check: Model is starting")
        return False
      else:
        print(f"Check: Model is in {state} state")
        return False
    else:
      print(f"Check: Model does not have deployment stats")
      return False

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.9/157.9 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.7 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/eland/ml/_optional.py:116: UserWarning: Eland requires version '1.3' or newer of 'sklearn' (version '1.2.2' currently installed). Use pip or conda to update sklearn.
  warnings.warn(msg, UserWarning)


### Elastic MLにE5モデルをアップロード

In [12]:
if enable_multilinguale5small:
    load_model(es, "intfloat/multilingual-e5-small", "text_embedding")

Loading HuggingFace transformer tokenizer and model [intfloat/multilingual-e5-small] for task [text_embedding]


tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

Model has already been imported


### Elastic ML E5モデルの開始

In [13]:
if enable_multilinguale5small:
    start_model(es, model_id="intfloat/multilingual-e5-small")


Check: Model does not have deployment stats
Starting model deployment 'intfloat__multilingual-e5-small'
Model successfully started with id 'intfloat__multilingual-e5-small'


### Elastic MLにTohoku Bertモデルをアップロード

In [14]:
if enable_bertbasejapanesev3:
    %pip install -q fugashi ipadic unidic_lite
    load_model(es, "cl-tohoku/bert-base-japanese-v3", "text_embedding")

if enable_bertbasejapanesev2:
    %pip install -q fugashi ipadic unidic_lite
    load_model(es, "cl-tohoku/bert-base-japanese-v2", "text_embedding")

### Elastic ML Tohoku Bertモデルの開始

In [15]:
if enable_bertbasejapanesev3:
    start_model(es, model_id="cl-tohoku/bert-base-japanese-v3")
if enable_bertbasejapanesev2:
    start_model(es, model_id="cl-tohoku/bert-base-japanese-v2")

## テスト結果をまとめる関数

In [19]:
import json

# Initialize a list to store the results


# Define the file path for data persistence
file_path = f"{INDEX_PREFIX}_results.json"
result_list = {}

def add_result(query, search_logic, result):
    global result_list
    # クエリが存在しない場合、新しいクエリを追加
    if query not in result_list["queries"]:
        result_list["queries"][query] = {}

    # ロジックが存在しない場合、新しいロジックを追加
    if search_logic not in result_list["queries"][query]:
        result_list["queries"][query][search_logic] = []

    # 結果を追加
    result_list["queries"][query][search_logic] = result
    save_results()

def save_results():
    global result_list
    with open(file_path, 'w') as file:
        json.dump(result_list, file, ensure_ascii=False, indent=2)

def load_results():
    global result_list
    try:
      if os.path.exists(file_path):
        with open(file_path, 'r', encoding='utf-8') as file:
            result_list = json.load(file)
      else:
        result_list = { "queries": {} }
    except FileNotFoundError:
        # If the file doesn't exist, start with an empty results list
        result_list.clear()

# Load existing results from the file (if any)
load_results()

# 2.検索ドキュメントのセットアップ


## Qiita記事のダウンロード

In [3]:
%pip install requests

import json
import requests
import os

# h = {'Authorization': 'Bearer xxxx'} # ユーザ認証する場合
h = {}
url = "https://qiita.com/api/v2/items?"

# tag別に記事をPAGEだけ繰り返し取得
query = "&query=org%3Aelasticsearch_japan"
# 検索で指定した期間内に作成された記事数を取得
res = requests.get(url=url + query, headers=h)
# サーバーからの応答
print(res.status_code, res.reason)
# print("指定しているタグ: " + tag_name)
total_count = int(res.headers['Total-Count'])
print("total_count: " + str(total_count))

page = f"page=1&per_page={total_count}"
os.makedirs("qiita/elasticsearch_japan", exist_ok=True)
res = requests.get(url=url + page + query, headers=h)
documents = json.loads(res.text)
for doc in documents:
    search_doc = {
        "title": doc["title"],
        "url": doc["url"],
        "body": doc["body"],
        "tags": doc["tags"]
    }
    title = doc["title"].replace('/', '_')
    filename = "./qiita/" + "elasticsearch_japan" + "/" + title + ".json"
    with open(filename, 'w') as f:
        json.dump(search_doc, f, indent=2, ensure_ascii=False)

200 OK
total_count: 69


## GPTによる質問の自動生成

In [51]:
from openai import OpenAI

def generate_questions(context):
  input_text='以下の情報に対する質問を10個作成してください。質問は以下に含まれる情報だけを使ってください。答えはJSON形式でquestionsキーのリストとして返してください。次のようなjson形式でお願いします。\n' + \
  '''
{
  "questions": [
    "ElasticsearchのRPMインストール後に表示されるパスワードをどのように管理すべきですか？",
    "Kibanaのネットワークインターフェースの設定を変更する目的は何ですか？",
  ]
}
''' + "\n\n" + context

  client = OpenAI()

  response = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    response_format={ "type": "json_object" },
    messages=[
      {"role": "system", "content": "You are a helpful assistant."},
      {"role": "user", "content": input_text},
    ]
  )

  response_content = response.choices[0].message.content
  # print(response_content)
  response_content = json.loads(response_content)
  return response_content
  # response_content = json.loads(response_content)
  # print("Received response:" + json.dumps(response, ensure_ascii=False))

In [52]:
# フォルダのパスを指定
blog_path = './qiita/elasticsearch_japan/'
questions_path = './qiita/questions/'
os.makedirs(questions_path, exist_ok=True)
documents = []
file_list=os.listdir(blog_path)
total = len(file_list)
for i, filename in enumerate(file_list):
    print(f"Generating file for {i}/{total}: {filename}")
    file_path = os.path.join(blog_path, filename)
    with open(file_path, 'r', encoding='utf-8') as f:
        try:
            data = json.load(f)
            page_content = data.get('body')
            if len(page_content) <= 100:
                print(f"Skipping because content length is short")
                continue

            questions = generate_questions(page_content[:10000])
            docs_per_question = []
            try:
                for q in questions['questions']:
                    if not isinstance(q, str):
                        raise Exception("bad format: " + data.get('title'))
                    doc = data.copy()
                    del doc['body']
                    doc['question'] = q
                    docs_per_question.append(doc)
            except Exception:
                print(f"Error at {file_path}")
                print(questions)
                continue

            new_file_name = filename
            new_file_path = os.path.join(questions_path, new_file_name)

            # "questions.json"としてデータを書き出す
            with open(new_file_path, 'w', encoding='utf-8') as new_file:
                json.dump(docs_per_question, new_file, ensure_ascii=False, indent=2)

        except json.JSONDecodeError as e:
            print(f'Error decoding JSON in file {file_path}: {e}')

Generating file for 0/69: AWS s3 をElasticsearchに取り込んで検索してみる（パート２: NLP Inferenceを追加）.json
Generating file for 1/69: Elastic Cloud について　〜実際にデプロイメントを作ってみよう〜.json
Generating file for 2/69: ElasticsearchでRAG (Retrieval Augmented Generation) を試す.json
Generating file for 3/69: GKE上のOpenTelemetry DemoアプリケーションをさくっとElasticでAPM・ログ・インフラ+eBPFプロファイリング監視する.json
Generating file for 4/69: Elastic Stackに対してTerraformのImportをやってみた.json
Generating file for 5/69: Elasticsearchにカスタム時系列データを取り込む (Elastic Agent編).json
Generating file for 6/69: Elastic Stack コードリーディング: Painless スクリプト.json
Generating file for 7/69: Elastic Cloudに対してTerraformのImportをやってみた.json
Generating file for 8/69: [v8.7] Elasticsearch multinode_dedicaded masterをdocker-composeでインストールする手順（試用用途）.json
Generating file for 9/69: Elastic Stack 8.0 の NLP で日本語センチメント分析を試してみた - 前編.json
Generating file for 10/69: APMでElasticsearchのクエリーをどこまで分析できるかやってみた.json
Generating file for 11/69: [v8.6] Elasticsearch_Kibanaをdocker-composeでインストールする手順（試用用途）.json
Generat

In [16]:
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import JSONLoader

DRIVE_FOLDER = "./qiita/elasticsearch_japan/"

def metadata_func(record: dict, metadata: dict) -> dict:

    metadata["title"] = record.get("title")
    metadata["url"] = record.get("url")
    metadata["tags"] = record.get("tags")

    return metadata

loader_kwargs = {
    'jq_schema':'.',
    'text_content': True,
    'content_key': "body",
    'metadata_func': metadata_func
}

loader = DirectoryLoader(DRIVE_FOLDER, glob='**/*.json', show_progress=True, loader_cls=JSONLoader, loader_kwargs = loader_kwargs)

json_docs = loader.load()

pprint(f'document count: {len(json_docs)}')
pprint(json_docs[0] if len(json_docs) > 0 else None)
pprint(json_docs[0].metadata if len(json_docs) > 0 else None)

100%|██████████| 69/69 [00:00<00:00, 140.30it/s]

'document count: 69'
Document(page_content='# はじめに\n前回の[パート1](https://qiita.com/nobuhikosekiya/items/f58a9c72d3972d3e24cc)ではAWS S3のバケットに入っているファイルをElasticの[S3 Connector Client](https://www.elastic.co/guide/en/enterprise-search/8.7/connectors-s3.html)を使ってElasticsearchにインデックスしてテキスト情報などを取り込みました。\n\n今回は、現在流行りのNLPの話題に乗っかるために、Elasticの[NLP Inference機能](https://www.elastic.co/guide/en/machine-learning/8.7/ml-nlp-inference.html)を試します。\n具体的に何ができるようになるかというと、Elasticsearchにインデックスされたデータ（今回はS3バケット内にアップロードされているファイルに含まれるテキスト情報）に対してM/L学習モデルを使ってInference（推論）をかけ、その出力を同じElasticsearchのドキュメントに保存（インデックス）できます。\nどのようなM/L学習モデルが使えるかは[こちら](https://www.elastic.co/guide/en/machine-learning/8.7/ml-nlp-model-ref.html)に書かれています。\n\n# MLノードの追加\n最初に、ElasticsearchのMLノードが必要です。Elastic Cloudの場合Machine Learning Nodeを数クリックで簡単にデプロイできてしまいます :)\n![image.png](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/109197/8f6c97b7-b99e-e7a4-0dc8-033372a8003f.png)\nとりあえず低めのスペック（２GB RAM)からスタートしています。あとで簡単に拡張できるので。\n\n# MLモデルのデプロイ\nサ

In [17]:
if False: #質問検索する場合はTrueに変えてください
    from langchain.document_loaders import DirectoryLoader
    from langchain.document_loaders import JSONLoader

    DRIVE_FOLDER = "./qiita/questions/"

    def metadata_func(record: dict, metadata: dict) -> dict:

        metadata["title"] = record.get("title")
        metadata["url"] = record.get("url")
        metadata["tags"] = record.get("tags")

        return metadata

    loader_kwargs = {
        'jq_schema':'.[]',
        'text_content': True,
        'content_key': "question",
        'metadata_func': metadata_func
    }

    loader = DirectoryLoader(DRIVE_FOLDER, glob='**/*.json', show_progress=True, loader_cls=JSONLoader, loader_kwargs = loader_kwargs)

    json_docs = loader.load()

    pprint(f'document count: {len(json_docs)}')
    pprint(json_docs[0] if len(json_docs) > 0 else None)

# 3.検索の質問の設定

In [18]:
questions = [
    "Frozen tierの使い方について教えてください",
]

## Run Allする際にストップしますか？

In [20]:
AUTO_RUN=input("Auto run the tests? Enter any key for true, no key for false") != ""
if not AUTO_RUN:
    raise UserWarning('Exit to stop here.')

Auto run the tests? Enter any key for true, no key for falsed


# 3.様々なサーチを試そう

## Elasticsearchのキーワード検索 (BM25)

### インデックス作成
Kuromojiをアナライザーとして設定したElasticsearchのインデックスを作成します。

In [21]:
INDEX_NAME=f"{INDEX_PREFIX}_bm25"
INDEX_NAME

'test1226_bm25'

In [22]:
INDEX_NAME=f"{INDEX_PREFIX}_bm25"
if es.indices.exists(index=INDEX_NAME):
    # If it exists, delete the index
    es.indices.delete(index=INDEX_NAME)
    print(f"Index '{INDEX_NAME}' deleted successfully.")
else:
    print(f"Index '{INDEX_NAME}' does not exist.")

es.indices.create(
  index=INDEX_NAME,
  settings={
      "index": {
          "number_of_shards": 1,
          "number_of_replicas": 0
      }
  }
)


Index 'test1226_bm25' deleted successfully.


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'test1226_bm25'})

### Kuromojiアナライザの設定

In [23]:
INDEX_NAME=f"{INDEX_PREFIX}_bm25"
es.indices.close(index=INDEX_NAME)

add_settings = {
  "index": {
    "analysis": {
      "char_filter": {
        "normalize": {
          "mode": "compose",
          "name": "nfkc",
          "type": "icu_normalizer"
        }
      }
    }
  }
}
es.indices.put_settings(index=INDEX_NAME, body=add_settings)

add_settings = {
  "index": {
    "analysis": {
      "filter": {
        "ja_index_synonym": {
          "type": "synonym",
          "lenient": "false",
          "synonyms": []
        }
      }
    }
  }
}
es.indices.put_settings(index=INDEX_NAME, body=add_settings)

add_settings = {
  "index": {
    "analysis": {
      "tokenizer": {
        "ja_kuromoji_tokenizer": {
          "mode": "search",
          "discard_compound_token": "true",
          "type": "kuromoji_tokenizer"
        }
      }
    }
  }
}

es.indices.put_settings(index=INDEX_NAME, body=add_settings)

# Define a synonym set
synonyms_set = [
    # {
    #   "id": "test-1",
    #   "synonyms": "foo, bar"
    # },
    # {
    #   "id": "test-2",
    #   "synonyms": "test => check"
    # }
  ]

# Define the identifier for the synonym set
synonym_id = "ja_search_synonym-set"

from elasticsearch.client import SynonymsClient
# Call the put_synonym function to add or update the synonym set
response = SynonymsClient(es).put_synonym(id=synonym_id, synonyms_set=synonyms_set)

# Define the new settings you want to apply
add_settings = {
  "index": {
    "analysis": {
      "filter": {
        "ja_search_synonym": {
          "type": "synonym_graph",
          "synonyms_set": "ja_search_synonym-set",
          "updateable": "true"
        }
      },
      "analyzer": {
        "ja_kuromoji_index_analyzer": {
          "filter": [
            "kuromoji_baseform",
            "kuromoji_part_of_speech",
            "ja_index_synonym",
            "cjk_width",
            "ja_stop",
            "kuromoji_stemmer",
            "lowercase"
          ],
          "char_filter": [
            "normalize"
          ],
          "type": "custom",
          "tokenizer": "ja_kuromoji_tokenizer"
        },
        "ja_kuromoji_search_analyzer": {
          "filter": [
            "kuromoji_baseform",
            "kuromoji_part_of_speech",
            "ja_search_synonym",
            "cjk_width",
            "ja_stop",
            "kuromoji_stemmer",
            "lowercase"
          ],
          "char_filter": [
            "normalize"
          ],
          "type": "custom",
          "tokenizer": "ja_kuromoji_tokenizer"
        }
      }
    }
  }
}

es.indices.put_settings(index=INDEX_NAME, body=add_settings)

es.indices.open(index=INDEX_NAME, request_timeout=60)


<ipython-input-23-77f437a2cdf9>:65: DeprecationWarning: Importing from the 'elasticsearch.client' module is deprecated. Instead use 'elasticsearch' module for importing the client.
  from elasticsearch.client import SynonymsClient
<ipython-input-23-77f437a2cdf9>:120: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.open(index=INDEX_NAME, request_timeout=60)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True})

### Mapping定義

In [24]:
INDEX_NAME=f"{INDEX_PREFIX}_bm25"
es.indices.close(index=INDEX_NAME, request_timeout=60)

# Define the new mapping
add_mapping = {
  "properties": {
    "page_content": {
      "type": "text",
      "search_analyzer": "ja_kuromoji_search_analyzer",
      "analyzer": "ja_kuromoji_index_analyzer"
    },
    "metadata": {
      "properties": {
        "seq_num": {
          "type": "long"
        },
        "source": {
          "type": "keyword"
        },
        "tags": {
          "properties": {
            "name": {
              "type": "text",
              "search_analyzer": "ja_kuromoji_search_analyzer",
              "analyzer": "ja_kuromoji_index_analyzer"
            }
          }
        },
        "title": {
          "type": "text",
          "search_analyzer": "ja_kuromoji_search_analyzer",
          "analyzer": "ja_kuromoji_index_analyzer"
        },
        "url": {
          "type": "text",
        }
      }
    }
  }
}

es.indices.put_mapping(index=INDEX_NAME, body=add_mapping)

es.indices.open(index=INDEX_NAME)

<ipython-input-24-b38c42e6f337>:2: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.close(index=INDEX_NAME, request_timeout=60)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True})

### インジェスト

In [25]:
INDEX_NAME=f"{INDEX_PREFIX}_bm25"
from elasticsearch import Elasticsearch, helpers

# 前の実行で残っているDocumentはクリアしてからインジェストします
if es.indices.exists(index=INDEX_NAME):
    es.delete_by_query(index=INDEX_NAME, body={"query": {"match_all": {}}})

docs_json = [doc.to_json()["kwargs"] for doc in json_docs]
# print(docs_json)

index_docs = []
for doc_json in docs_json:
    index_docs.append({
        "_index": INDEX_NAME,
        "_source": doc_json,
    })

helpers.bulk(es, index_docs)
es.indices.refresh(index=INDEX_NAME)

response = es.search(index=INDEX_NAME, query={"match_all": {}}, fields=["metadata"])
for hit in response['hits']['hits']:
    metadata = hit['_source']['metadata']
    print(f"metadata: {metadata}")

metadata: {'source': '/content/qiita/elasticsearch_japan/AWS s3 をElasticsearchに取り込んで検索してみる（パート２: NLP Inferenceを追加）.json', 'seq_num': 1, 'title': 'AWS s3 をElasticsearchに取り込んで検索してみる（パート２: NLP Inferenceを追加）', 'url': 'https://qiita.com/nobuhikosekiya/items/765a7269da8809898936', 'tags': [{'name': 'S3', 'versions': []}, {'name': 'NLP', 'versions': []}, {'name': 'Elasticsearch', 'versions': []}]}
metadata: {'source': '/content/qiita/elasticsearch_japan/Elastic Cloud について\u3000〜実際にデプロイメントを作ってみよう〜.json', 'seq_num': 1, 'title': 'Elastic Cloud について\u3000〜実際にデプロイメントを作ってみよう〜', 'url': 'https://qiita.com/tomo_s_el/items/3584d0b1fabb0bafa4fa', 'tags': [{'name': 'Elasticsearch', 'versions': []}, {'name': 'elasticcloud', 'versions': []}]}
metadata: {'source': '/content/qiita/elasticsearch_japan/ElasticsearchでRAG (Retrieval Augmented Generation) を試す.json', 'seq_num': 1, 'title': 'ElasticsearchでRAG (Retrieval Augmented Generation) を試す', 'url': 'https://qiita.com/takeo-furukubo/items/e5d43fa734e4338b895f'

### サーチ

In [26]:
INDEX_NAME=f"{INDEX_PREFIX}_bm25"
def do_search(query_text, filter=None, subfield=None):
    subfield="prefix"
    query = {
        "bool": {
            "must": [
                {
                    "match": {
                        "page_content": {
                            "query": query_text,
                            "analyzer": "ja_kuromoji_search_analyzer"
                        }
                    }
                }
            ]
        }
    }

    # フィルタが指定されている場合はフィルタを追加
    if filter is not None:
        query["bool"]["filter"] = filter

    highlight = {
        "fields": {
            "page_content": {}
        }
    }
    fields = ["metadata"]
    response = es.search(index=INDEX_NAME, query=query, fields=fields, highlight=highlight)

    results = []
    # Iterate through the search results and access the fields you want
    for hit in response['hits']['hits']:
        metadata = hit['_source']['metadata']
        score = hit['_score']
        highlight = hit['highlight']
        print(f"_score: {score}, metadata: {metadata}")
        pprint(f"{highlight}")
        r = {
            "metadata": metadata,
            "highlight": highlight
        }
        results.append(json.dumps(r, ensure_ascii=False))

    return {"query": query_text, "result": results}

In [27]:
INDEX_NAME=f"{INDEX_PREFIX}_bm25"
for query in questions:
    print(query)
    result = do_search(query)
    add_result(search_logic="bm25", query=query, result=result)

Frozen tierの使い方について教えてください
_score: 10.334702, metadata: {'source': '/content/qiita/elasticsearch_japan/Elastic Cloud について\u3000〜実際にデプロイメントを作ってみよう〜.json', 'seq_num': 1, 'title': 'Elastic Cloud について\u3000〜実際にデプロイメントを作ってみよう〜', 'url': 'https://qiita.com/tomo_s_el/items/3584d0b1fabb0bafa4fa', 'tags': [{'name': 'Elasticsearch', 'versions': []}, {'name': 'elasticcloud', 'versions': []}]}
("{'page_content': ['- クラスターのデプロイ\\n    - Elasticsearch ノード (Hot, Warm, Cold, "
 "<em>Frozen</em>, Machine Learning, Coordinating)\\n    -', '[[Pricing "
 'Calculator]](https://cloud.elastic.co/pricing/) '
 "というツールですが、構成をポチポチと選択すると、その金額を<em>教え</em>てくれるので金額感やサイズ', '```\\n- Hot "
 '<em>tier</em> node 8GB RAM x 2zones\\n- Master (tie breaker) node 1GB RAM x '
 "1zone (自動追加)\\n- Integration', '＊ただし、Machine Learning 自体はplatinum "
 "ライセンスの機能なのでご注意<em>ください</em>。', "
 "'少し長くなったので、いったんここで区切りましたが、まだまだ色々と便利な機能があるので、<em>使い方</em>や tips "
 "含め書いていきたいと思います。']}")
_score: 8.751479, metadata: {'source': '/content/qiita/elastic

## （Open AI Embeddingを利用） Elasticsearchのセマンティック検索
LangchainのElasticsearchStoreというクラスを使って色々なAI/MLモデルを切り替えながらElasticsearchのベクトル検索をテストしていくことができます。

参考：https://python.langchain.com/docs/integrations/vectorstores/elasticsearch

### インジェスト
補足：ElasticsearchStoreの中で自動的にインデックスの作成も行われています。

In [28]:
INDEX_NAME=f"{INDEX_PREFIX}_openai"
INDEX_NAME

'test1226_openai'

In [29]:
INDEX_NAME=f"{INDEX_PREFIX}_openai"

if os.environ["OPENAI_API_KEY"]  == "":
    print("Skipping because model is not enabled")
else:
    if USE_AZURE_OPENAI:
        embedding = OpenAIEmbeddings(deployment=azureOpenAI_embedding_deployment)
    else:
        embedding = OpenAIEmbeddings(openai_api_type="open_ai")


    db_openai = ElasticsearchStore(
        es_connection=es,
        index_name=INDEX_NAME,
        embedding=embedding,
        strategy=ElasticsearchStore.ApproxRetrievalStrategy()
    )

In [30]:
if os.environ["OPENAI_API_KEY"]  == "":
    print("Skipping because model is not enabled")
else:

    # 前の実行で残っているDocumentはクリアしてからインジェストします
    if es.indices.exists(index=INDEX_NAME):
        db_openai.client.delete_by_query(index=INDEX_NAME, body={"query": {"match_all": {}}})
    db_openai.add_documents(json_docs)
    db_openai.client.indices.refresh(index=INDEX_NAME)

### サーチ

In [31]:
if os.environ["OPENAI_API_KEY"]  == "":
    print("Skipping because model is not enabled")
else:

    # for query in questions:
    for query in questions:
        print(query)
        results = db_openai.similarity_search_with_score(query, k=5)
        [print(score, element.metadata, element.page_content[:300]) for element, score in results]
        add_result(search_logic="vector_openai_embeddings", query=query, result=[(f"score: {score}", element.metadata, element.page_content[:100]) for element, score in results])

Frozen tierの使い方について教えてください
0.92430925 {'source': '/content/qiita/elasticsearch_japan/ElasticsearchのFrozenデータティアにデータが入るのをテストしてみた (1).json', 'seq_num': 1, 'title': 'ElasticsearchのFrozenデータティアにデータが入るのをテストしてみた (1)', 'url': 'https://qiita.com/nobuhikosekiya/items/dd2ce836b184730f8d70', 'tags': [{'name': 'Elasticsearch', 'versions': []}]} シリーズ
[ElasticsearchのFrozenデータティアにデータが入るのをテストしてみた (1)](https://qiita.com/nobuhikosekiya/items/dd2ce836b184730f8d70)
[ElasticsearchのFrozenデータティアにデータが入るのをテストしてみた (2)](https://qiita.com/nobuhikosekiya/items/8ab82e70953b6b5d1736)

# はじめに
こんにちは。Elasticのソリューションアーキテクトをしている関屋です。
Elasticsearchバージョン7.12からFroze
0.910676 {'source': '/content/qiita/elasticsearch_japan/ElasticsearchのFrozenデータティアにデータが入るのをテストしてみた (2).json', 'seq_num': 1, 'title': 'ElasticsearchのFrozenデータティアにデータが入るのをテストしてみた (2)', 'url': 'https://qiita.com/nobuhikosekiya/items/8ab82e70953b6b5d1736', 'tags': [{'name': 'Elasticsearch', 'versions': []}]} シリーズ
[ElasticsearchのFrozenデータティアにデータが入るのをテストしてみた (1)](http

## (ElasticsearchにアップしたE5モデルを利用） Elasticsearchのセマンティック検索

### インジェスト

In [32]:
INDEX_NAME=f"{INDEX_PREFIX}_esml_e5"
INDEX_NAME

'test1226_esml_e5'

In [33]:
INDEX_NAME=f"{INDEX_PREFIX}_esml_e5"
if not is_model_started(es, model_id="intfloat/multilingual-e5-small"):
    print("Skipping because model is not enabled")
else:

    from langchain.vectorstores.elasticsearch import ElasticsearchStore
    from langchain.embeddings.elasticsearch import ElasticsearchEmbeddings

    ES_MODEL_ID=elasticsearch_model_id("intfloat/multilingual-e5-small")

    embedding = ElasticsearchEmbeddings.from_es_connection(
        es_connection=es,
        model_id=ES_MODEL_ID
    )

    db_esml_e5 = ElasticsearchStore(
        es_connection=es,
        index_name=INDEX_NAME,
        embedding=embedding,
        strategy=ElasticsearchStore.ApproxRetrievalStrategy()
    )

Check: Model is started


In [34]:
INDEX_NAME=f"{INDEX_PREFIX}_esml_e5"
if not is_model_started(es, model_id="intfloat/multilingual-e5-small"):
    print("Skipping because model is not enabled")
else:

    if db_esml_e5.client.indices.exists(index=INDEX_NAME):
        db_esml_e5.client.delete_by_query(index=INDEX_NAME, body={"query": {"match_all": {}}})

    for doc in json_docs:
        db_esml_e5.add_documents([doc])
        print('.', end='')

    db_esml_e5.client.indices.refresh(index=INDEX_NAME)

Check: Model is started
.....................................................................

### サーチ

In [35]:
INDEX_NAME=f"{INDEX_PREFIX}_esml_e5"
if not is_model_started(es, "intfloat__multilingual-e5-small"):
    print("Skipping because model is not enabled")
else:

    # for query in questions:
    for query in questions:
        print(query)
        results = db_esml_e5.similarity_search_with_score(query, k=5)
        [print(score, element.metadata, element.page_content[:300]) for element, score in results]
        add_result(search_logic="vector_elastic_e5", query=query, result=[(f"score: {score}", element.metadata, element.page_content[:100]) for element, score in results])

Check: Model is started
Frozen tierの使い方について教えてください
0.95075536 {'source': '/content/qiita/elasticsearch_japan/ElasticsearchのFrozenデータティアにデータが入るのをテストしてみた (1).json', 'seq_num': 1, 'title': 'ElasticsearchのFrozenデータティアにデータが入るのをテストしてみた (1)', 'url': 'https://qiita.com/nobuhikosekiya/items/dd2ce836b184730f8d70', 'tags': [{'name': 'Elasticsearch', 'versions': []}]} シリーズ
[ElasticsearchのFrozenデータティアにデータが入るのをテストしてみた (1)](https://qiita.com/nobuhikosekiya/items/dd2ce836b184730f8d70)
[ElasticsearchのFrozenデータティアにデータが入るのをテストしてみた (2)](https://qiita.com/nobuhikosekiya/items/8ab82e70953b6b5d1736)

# はじめに
こんにちは。Elasticのソリューションアーキテクトをしている関屋です。
Elasticsearchバージョン7.12からFroze
0.942831 {'source': '/content/qiita/elasticsearch_japan/ElasticsearchのFrozenデータティアにデータが入るのをテストしてみた (2).json', 'seq_num': 1, 'title': 'ElasticsearchのFrozenデータティアにデータが入るのをテストしてみた (2)', 'url': 'https://qiita.com/nobuhikosekiya/items/8ab82e70953b6b5d1736', 'tags': [{'name': 'Elasticsearch', 'versions': []}]} シリーズ
[ElasticsearchのFrozenデータティアにデ

## Elasticsearch RRF ハイブリッド検索

### インジェスト

In [36]:
INDEX_NAME=f"{INDEX_PREFIX}_esml_e5_hybrid"
INDEX_NAME

'test1226_esml_e5_hybrid'

In [37]:
INDEX_NAME=f"{INDEX_PREFIX}_esml_e5_hybrid"
if not is_model_started(es, model_id="intfloat/multilingual-e5-small"):
    print("Skipping because model is not enabled")
else:

    from langchain.vectorstores.elasticsearch import ElasticsearchStore
    from langchain.embeddings.elasticsearch import ElasticsearchEmbeddings

    ES_MODEL_ID=elasticsearch_model_id("intfloat/multilingual-e5-small")

    embedding = ElasticsearchEmbeddings.from_es_connection(
        es_connection=es,
        model_id=ES_MODEL_ID
    )

    db_esml_e5_hybrid = ElasticsearchStore(
        es_connection=es.options(request_timeout=3600),
        index_name=INDEX_NAME,
        embedding=embedding,
        strategy=ElasticsearchStore.ApproxRetrievalStrategy(hybrid=True)
    )

Check: Model is started


In [38]:
INDEX_NAME=f"{INDEX_PREFIX}_esml_e5_hybrid"
if not is_model_started(es, "intfloat__multilingual-e5-small"):
    print("Skipping because model is not enabled")
else:

    if db_esml_e5_hybrid.client.indices.exists(index=INDEX_NAME):
        db_esml_e5_hybrid.client.delete_by_query(index=INDEX_NAME, body={"query": {"match_all": {}}})

    for doc in json_docs:
        db_esml_e5_hybrid.add_documents([doc])
        print(".", end="")
    db_esml_e5_hybrid.client.indices.refresh(index=INDEX_NAME)

Check: Model is started
.....................................................................

### サーチ

In [39]:
INDEX_NAME=f"{INDEX_PREFIX}_esml_e5_hybrid"
if not is_model_started(es, "intfloat__multilingual-e5-small"):
    print("Skipping because model is not enabled")
else:

    for query in questions:
        print(query)
        results = db_esml_e5_hybrid.similarity_search_with_score(query, k=5)
        [print(score, element.metadata, element.page_content[:300]) for element, score in results]
        add_result(search_logic="hybrid_elastic_bm25_e5", query=query, result=[(f"score: {score}", element.metadata, element.page_content[:100]) for element, score in results])

Check: Model is started
Frozen tierの使い方について教えてください
None {'source': '/content/qiita/elasticsearch_japan/ElasticsearchのFrozenデータティアにデータが入るのをテストしてみた (1).json', 'seq_num': 1, 'title': 'ElasticsearchのFrozenデータティアにデータが入るのをテストしてみた (1)', 'url': 'https://qiita.com/nobuhikosekiya/items/dd2ce836b184730f8d70', 'tags': [{'name': 'Elasticsearch', 'versions': []}]} シリーズ
[ElasticsearchのFrozenデータティアにデータが入るのをテストしてみた (1)](https://qiita.com/nobuhikosekiya/items/dd2ce836b184730f8d70)
[ElasticsearchのFrozenデータティアにデータが入るのをテストしてみた (2)](https://qiita.com/nobuhikosekiya/items/8ab82e70953b6b5d1736)

# はじめに
こんにちは。Elasticのソリューションアーキテクトをしている関屋です。
Elasticsearchバージョン7.12からFroze
None {'source': '/content/qiita/elasticsearch_japan/Elastic CloudでHotに溜まっているデータを別のティアに移動するやり方.json', 'seq_num': 1, 'title': 'Elastic CloudでHotに溜まっているデータを別のティアに移動するやり方', 'url': 'https://qiita.com/nobuhikosekiya/items/3c03932c3efec0a9f04d', 'tags': [{'name': 'Elasticsearch', 'versions': []}]} # はじめに
Elasticソリューションアーキテクトの関屋です。
Elastic Cloudでは、データを

## (ElasticsearchにアップしたTohoku BERT Japanese v3モデルを利用) Elasticのセマンティック検索

### インジェスト

In [40]:
INDEX_NAME=f"{INDEX_PREFIX}_esml_tohokubertv3"
INDEX_NAME

'test1226_esml_tohokubertv3'

In [41]:
INDEX_NAME=f"{INDEX_PREFIX}_esml_tohokubertv3"
if not is_model_started(es, model_id="cl-tohoku/bert-base-japanese-v3"):
    print("Skipping because model is not enabled")
else:

    from langchain.vectorstores.elasticsearch import ElasticsearchStore
    from langchain.embeddings.elasticsearch import ElasticsearchEmbeddings

    ES_MODEL_ID=elasticsearch_model_id("cl-tohoku/bert-base-japanese-v3")

    embedding = ElasticsearchEmbeddings.from_es_connection(
        es_connection=es,
        model_id=ES_MODEL_ID
    )

    db_esml_tohokubertv3 = ElasticsearchStore(
        es_connection=es,
        index_name=INDEX_NAME,
        embedding=embedding,
        strategy=ElasticsearchStore.ApproxRetrievalStrategy()
    )

Check: Model does not have deployment stats
Skipping because model is not enabled


In [42]:
INDEX_NAME=f"{INDEX_PREFIX}_esml_tohokubertv3"
if not is_model_started(es, "cl-tohoku__bert-base-japanese-v3"):
    print("Skipping because model is not enabled")
else:

    if db_esml_tohokubertv3.client.indices.exists(index=INDEX_NAME):
        db_esml_tohokubertv3.client.delete_by_query(index=INDEX_NAME, body={"query": {"match_all": {}}})
    for doc in json_docs:
        db_esml_tohokubertv3.add_documents([doc])
        print(".", end='')
    db_esml_tohokubertv3.client.indices.refresh(index=INDEX_NAME)

Check: Model does not have deployment stats
Skipping because model is not enabled


### サーチ

In [43]:
INDEX_NAME=f"{INDEX_PREFIX}_esml_tohokubertv3"
if not is_model_started(es, "cl-tohoku__bert-base-japanese-v3"):
    print("Skipping because model is not enabled")
else:

    for query in questions:
        print(query)

        results = db_esml_tohokubertv3.similarity_search_with_score(query, k=10)
        [print(score, element.metadata, element.page_content[:300]) for element, score in results]
        add_result(search_logic="esml_tohokubertv3", query=query, result=[(f"score: {score}", element.metadata, element.page_content[:100]) for element, score in results])

Check: Model does not have deployment stats
Skipping because model is not enabled


## (ElasticsearchにアップしたTohoku BERT Japanese v2モデルを利用) Elasticのセマンティック検索

### インジェスト

In [44]:
INDEX_NAME=f"{INDEX_PREFIX}_esml_tohokubertv2"
INDEX_NAME

'test1226_esml_tohokubertv2'

In [45]:
INDEX_NAME=f"{INDEX_PREFIX}_esml_tohokubertv2"
if not is_model_started(es, model_id="cl-tohoku/bert-base-japanese-v2"):
    print("Skipping because model is not enabled")
else:

    from langchain.vectorstores.elasticsearch import ElasticsearchStore
    from langchain.embeddings.elasticsearch import ElasticsearchEmbeddings

    ES_MODEL_ID=elasticsearch_model_id("cl-tohoku/bert-base-japanese-v2")

    embedding = ElasticsearchEmbeddings.from_es_connection(
        es_connection=es,
        model_id=ES_MODEL_ID
    )

    db_esml_tohokubertv2 = ElasticsearchStore(
        es_connection=es,
        index_name=INDEX_NAME,
        embedding=embedding,
        strategy=ElasticsearchStore.ApproxRetrievalStrategy()
    )

Check: Model is started


In [46]:
INDEX_NAME=f"{INDEX_PREFIX}_esml_tohokubertv2"
if not is_model_started(es, "cl-tohoku__bert-base-japanese-v2"):
    print("Skipping because model is not enabled")
else:

    if db_esml_tohokubertv2.client.indices.exists(index=INDEX_NAME):
        db_esml_tohokubertv2.client.delete_by_query(index=INDEX_NAME, body={"query": {"match_all": {}}})
    for doc in json_docs:
        db_esml_tohokubertv2.add_documents([doc])
        print(".", end='')
    db_esml_tohokubertv2.client.indices.refresh(index=INDEX_NAME)

Check: Model is started
.....................................................................

### サーチ

In [47]:
INDEX_NAME=f"{INDEX_PREFIX}_esml_tohokubertv2"
if not is_model_started(es, "cl-tohoku__bert-base-japanese-v2"):
    print("Skipping because model is not enabled")
else:

    for query in questions:
        print(query)

        results = db_esml_tohokubertv2.similarity_search_with_score(query, k=10)
        [print(score, element.metadata, element.page_content[:300]) for element, score in results]
        add_result(search_logic="esml_tohokubertv2", query=query, result=[(f"score: {score}", element.metadata, element.page_content[:100]) for element, score in results])

Check: Model is started
Frozen tierの使い方について教えてください
0.7157447 {'source': '/content/qiita/elasticsearch_japan/delika のデータを Elastic Stack で分析じゃ.json', 'seq_num': 1, 'title': 'delika のデータを Elastic Stack で分析じゃ', 'url': 'https://qiita.com/ijokarumawak@github/items/2d9022c59b34cc89fab8', 'tags': [{'name': 'Elasticsearch', 'versions': []}, {'name': 'Kibana', 'versions': []}, {'name': 'delika', 'versions': []}, {'name': 'Qiitadelika', 'versions': []}]} データの Github になる！を目指す [delika](https://delika.io/) 。お恥ずかしながら今まで存じ上げず、ふと Qiita で何か書こうかな、と眺めていたら Qiita で delika の記事投稿キャンペーンをやっていたので、やってみました！

この記事で利用している Elastic Stack は ver 8.1.0 です。

## まずはデータを取り込み！

delika には creative commons などでライセンスされたパブリックなデータセットが公開されています。さっそくいくつか取り込んでみましょう。

## 品川区避難所

CC 2.1 JP
0.7097299 {'source': '/content/qiita/elasticsearch_japan/[小ネタ] ElasticのAPMでコードに手を入れずにメソッド監視する方法.json', 'seq_num': 1, 'title': '[小ネタ] ElasticのAPMでコードに手を入れずにメソッド監視する方法', 'url': 'https://qiita.com/nobuhikosekiya/items/5dd6d910aec83dbe3f15', 'tags': [{'na

# 終了処理

## Elastic ML E5モデルの停止

In [48]:
if is_model_started(es, model_id="intfloat/multilingual-e5-small"):
    stop_model(es, model_id="intfloat/multilingual-e5-small")

Check: Model is started
Stopping model deployment


<ipython-input-11-17415ad073d6>:38: ElasticsearchWarning: The default [remove_binary] value of 'false' is deprecated and will be set to 'true' in a future release. Set [remove_binary] explicitly to 'true' or 'false' to ensure no behavior change.
  es_connection.options(request_timeout=300).ml.stop_trained_model_deployment(model_id=es_model_id, force=True)


## Elastic ML Tohoku BERT モデルの停止

In [49]:
if is_model_started(es, model_id="cl-tohoku/bert-base-japanese-v3"):
    stop_model(es, model_id="cl-tohoku/bert-base-japanese-v3")
if is_model_started(es, model_id="cl-tohoku/bert-base-japanese-v2"):
    stop_model(es, model_id="cl-tohoku/bert-base-japanese-v2")

Check: Model does not have deployment stats
Check: Model is started
Stopping model deployment


<ipython-input-11-17415ad073d6>:38: ElasticsearchWarning: The default [remove_binary] value of 'false' is deprecated and will be set to 'true' in a future release. Set [remove_binary] explicitly to 'true' or 'false' to ensure no behavior change.
  es_connection.options(request_timeout=300).ml.stop_trained_model_deployment(model_id=es_model_id, force=True)
